In [ ]:
import course;course.header()

# Days 3
## Overview

In [ ]:
course.display_topics(day=3)

# Discssion @ Excersises 



# A building block for excersises 1 & 2

In [ ]:
seqs = {}
current_id = None
current_seq = ""
with open("../data/uniprot-filtered-organism__Homo+sapiens+(Human)+[9606]_+AND+review--.fasta") as fasta_file:
    for line in fasta_file:
        if line.startswith(">"):
            if current_id is not None:
                seqs[current_id] = current_seq
            current_seq = ""
            current_id = line.strip()
        else:
            current_seq += line.strip()
    seqs[current_id] = current_seq
    

In [ ]:
!ls ../data

# Eval building block for excersise 2

In [ ]:
longest = None
longest_sequence = -1 
for identifier, sequence in seqs.items():
    if len(sequence) > longest_sequence:
        longest = identifier
        longest_sequence = len(sequence)

In [ ]:
longest

Say hello to [Titin](https://en.wikipedia.org/wiki/Titin) /Taitin/ 

# Plotting with plotly

Plotly is an interactive plotting library supporting many programming languages. Plotlys plots are based on javascript build on top of d3.js and stack.gl.

Advantages comapred to ggplot, matplotlib (and derivitates, seaborn ..), bokeh, pygal are:
* interactive (mouse over, zoom ...)
* directly embeddable as html
* realtime plots

In [ ]:
# basic setup
import plotly
import plotly.graph_objs as go
plotly.__version__

In [ ]:
import pandas as pd
pd.__version__

In [ ]:
# reading our small amino acid data set
# we will discuss pandas in the second week ...
aa_df = pd.read_csv("../data/amino_acid_properties.csv")

In [ ]:
aa_df.head()

# Bar chart

In [ ]:
import plotly.graph_objects as go

data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pka1
    )
]

fig = go.Figure(data=data)
fig.show()


Yes, not looking too nice ...

In [ ]:
fig.update_layout(template="seaborn")
fig.show()


Plotly comes with a set up templates, yet you can customize every bit of it ...

In [ ]:
import plotly.io as pio
pio.templates

In [ ]:
import plotly.graph_objects as go

data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df["hydropathy index (Kyte-Doolittle method)"]
    )
]

fig = go.Figure(data=data)
fig.show()



Let define amino acids based on hydropathy level into three categories
* hp < -2 : "hydrophilic"
* -2 < hp < 2 : "neutral"
* hp > 2 : "hydrophobic"

In [ ]:
aa_df['hp_type'] = "neutral"
aa_df.loc[aa_df['hydropathy index (Kyte-Doolittle method)'] < -2, 'hp_type'] = 'hydrophilic'
aa_df.loc[aa_df['hydropathy index (Kyte-Doolittle method)'] > 2, 'hp_type'] = 'hydrophobic'

In [ ]:
aa_df

How to map hp_types to colors?

In [ ]:
aa_df['hp_type'].map({'hydrophilic': "blue", 'neutral': "green", "hydrophobic": "black"})

In [ ]:

data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df["hydropathy index (Kyte-Doolittle method)"],
        marker_color=aa_df['hp_type'].map(
            {
                "hydrophilic": "blue", 
                "neutral": "green", 
                "hydrophobic": "white"
            }
        )
    )
]

fig = go.Figure(data=data)
fig.update_layout(template="plotly_dark", title="AA hydropathy index")

fig.show()



Lets add more information to the hovertext!

In [ ]:

data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df["hydropathy index (Kyte-Doolittle method)"],
        marker_color=aa_df['hp_type'].map(
            {
                "hydrophilic": "blue", 
                "neutral": "green", 
                "hydrophobic": "white"
            }
        ),
        text="Name:" + aa_df['Name'] + "<br />" +\
            "Weight:" + aa_df['Residue Weight'].astype(str) + "<br />" +\
            "Formula:" + aa_df['Residue Formula']
            

    )
]

fig = go.Figure(data=data)
fig.update_layout(template="plotly_dark", title="AA hydropathy index")

fig.show()



Let's sort and group!

In [ ]:
aa_df.sort_values(["hydropathy index (Kyte-Doolittle method)"], inplace=True)

In [ ]:
data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df["hydropathy index (Kyte-Doolittle method)"],
        marker_color=aa_df['hp_type'].map(
            {
                "hydrophilic": "blue", 
                "neutral": "green", 
                "hydrophobic": "white"
            }
        ),
        text="Name:" + aa_df['Name'] + "<br />" +\
            "Weight:" + aa_df['Residue Weight'].astype(str) + "<br />" +\
            "Formula:" + aa_df['Residue Formula']
            

    )
]

fig = go.Figure(data=data)
fig.update_layout(template="plotly_dark", title="AA hydropathy index")

fig.show()


Group data into legendgroups

In [ ]:
data = []
for hp_type in aa_df['hp_type'].unique():
    selected_df = aa_df[aa_df['hp_type'] == hp_type]
    data.append(
        go.Bar(
            x=selected_df["1-letter code"],
            y=selected_df["hydropathy index (Kyte-Doolittle method)"],
            legendgroup=hp_type,
            name=hp_type,
            
        )
    )

fig = go.Figure(data=data)
fig.update_layout(template="plotly_dark", title="Amino acid accessible surface")
fig.show()

## Pie chart

In [ ]:
import plotly.graph_objects as go

data = [
    go.Pie(
        labels=aa_df["1-letter code"],
        values=aa_df["Accessible surface"],
        hole=0.3,
#         pull=aa_df["hydropathy index (Kyte-Doolittle method)"].abs() / aa_df["hydropathy index (Kyte-Doolittle method)"].max() 
    )
]

fig = go.Figure(data=data)
fig.update_layout(template="plotly_dark", title="Amino acid accessible surface")
fig.show()



# some other styles - encoding more data

In [ ]:
import numpy as np
data = [
    go.Scatter(
        x=aa_df["1-letter code"],
        y=aa_df["pI"],
        mode="markers",
        marker={
            "size": aa_df["Accessible surface"],
            "color":aa_df['hp_type'].map(
                {
                    "hydrophilic": "blue", 
                    "neutral": "green", 
                    "hydrophobic": "white"
                }
            ),
        }
    )
]

fig = go.Figure(data=data)
fig.show()

In [ ]:
import plotly.graph_objects as go

data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pka1
    )
]

fig = go.Figure(
    data=data,
    layout={
        "title": {
            "text": "amino acid pka1's",
            "font_size": 30
        }
    }
)
# fig.show()


In [ ]:
# works also with dot annotations
data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pka1
    ),
#     go.Scatter(
#         x=aa_df["1-letter code"],
#         y=aa_df.pka1
#     )
]

fig = go.Figure(data=data)
fig.layout.title.text = "amino acid pka1's"
fig.layout.title.font.size = 40
fig.show()
# not quite consistent and api is still evolving

In [ ]:
# works also with dot annotations
data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pka1
    )
]

fig = go.Figure(data=data)
fig.layout.title.text = "amino acid pka1's"
fig.layout.title.font.size = 40
fig.data[0].marker.line.width = 5
fig.data[0].marker.line.color = "black"
fig.show()
# not quite consistent and api is still evolving

In [ ]:
# works also with dot annotations
data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pka1,
        hovertext=aa_df["Name"]
    )
]

fig = go.Figure(data=data)
fig.layout.title.text = "amino acid pka1's"
fig.layout.title.font.size = 30
fig.data[0].marker.line.width = 5
fig.data[0].marker.line.color = "black"
fig.show()
# not quite consistent and api is still evolving

In [ ]:
# pio.templates.default = "plotly"

data = [
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pka1,
        hovertext=aa_df["Name"],
        name="pka1"
    ),
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pka2,
        hovertext=aa_df["Name"],
        name="pka2"
    ),
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pkaX,
        hovertext=aa_df["Name"],
        name="pkaX"
    ),
    go.Bar(
        x=aa_df["1-letter code"],
        y=aa_df.pI,
        hovertext=aa_df["Name"],
        name="pI"
    )

]

In [ ]:
layout = {
    "title": {
        "text": "amino acid pks",
        "font_family": "Courier",
        "font_size": 30,
    },
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "yaxis": {
        "showgrid": True,
        "gridwidth": 1,
        "gridcolor": "rgba(0, 0, 0, 0.2)",
        "color": "rgba(0,0,0,1)",
        "ticks": "outside",
        "tickvals": [1, 5, 10],
        "title": {
            "text": "pka",
            "font_family": "Courier",
        },
        "showline": True,
        "linewidth": 1,
        "linecolor": "black",
        "mirror": True,
#         "type": "log"
    }
}

fig = go.Figure(data=data, layout=layout)
fig.update_xaxes(showline=True, linewidth=1, linecolor="black")
fig.show()

more details about axis can be found [here](https://plot.ly/python/axes/)

Plot examples can be found [here](https://plot.ly/python/)

## Questions ?

## Let's plot something

Anyone go ahead, take the amino_acid_visualization and plot something!